In [2]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time
from math import exp

#======================================#
# Pure Torch PINN with exact BCs
# (architecture & BC same as your first code)
#======================================#

class PINN(nn.Module):
    def __init__(self, flag_bc_exact=False):
        super(PINN, self).__init__()
        self.flag_bc_exact = flag_bc_exact

        # CHANGE: Restored the original architecture (32→128→64) like your first code
        self.layer = nn.Sequential(
            nn.Linear(1, 32),
            nn.SiLU(),
            nn.Linear(32, 128),
            nn.SiLU(),
            nn.Linear(128, 64),
            nn.SiLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out = self.layer(x)
        if self.flag_bc_exact:
            # CHANGE: same BC enforcement formula as first code
            # C(0)=1.0, C(1)=0.1  (change if BCs change!)
            out = out * x * (x - 1) + (-0.9 * x + 1.0)
        return out

# -------------------------------
# Loss: PDE Residual
# -------------------------------
def pde_residual(model, x, diff, vel, device):
    # CHANGE: create fresh tensor every epoch with requires_grad=True
    x_tensor = torch.tensor(x, dtype=torch.float32, device=device, requires_grad=True)
    diff_tensor = torch.tensor(diff, dtype=torch.float32, device=device)
    vel_tensor = torch.tensor(vel, dtype=torch.float32, device=device)

    # CHANGE: use x_tensor inside model, not x directly
    c = model(x_tensor)
    c_x = torch.autograd.grad(c, x_tensor, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    c_xx = torch.autograd.grad(c_x, x_tensor, grad_outputs=torch.ones_like(c_x), create_graph=True)[0]

    residual = vel_tensor * c_x - diff_tensor * c_xx
    loss = nn.MSELoss()(residual, torch.zeros_like(residual))
    return loss

# -------------------------------
# Loss: Boundary Conditions
# -------------------------------
def boundary_loss(model, xb, cb, device):
    # (unchanged)
    xb_tensor = torch.tensor(xb, dtype=torch.float32, device=device)
    cb_tensor = torch.tensor(cb, dtype=torch.float32, device=device)
    out = model(xb_tensor)
    return nn.MSELoss()(out, cb_tensor)

# -------------------------------
# Training Function
# -------------------------------
def pinn_train(x, xb, cb, vel, diff, batch_size, epochs=2000, flag_bc_exact=False, device="cpu"):
    # CHANGE: ensure model uses new architecture & BC enforcement
    model = PINN(flag_bc_exact=flag_bc_exact).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    tic = time.time()
    for epoch in range(epochs):
        model.train()

        # CHANGE: call pde_residual and boundary_loss like first code
        loss_eqn = pde_residual(model, x, diff, vel, device)
        loss_bc = boundary_loss(model, xb, cb, device)

        # CHANGE: same conditional loss selection as first code
        loss = loss_eqn if flag_bc_exact else (loss_eqn + loss_bc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6e}")

    print("Elapsed time =", time.time() - tic, "seconds")
    return model

# -------------------------------
# Main execution
# -------------------------------
if __name__ == "__main__":
    # Parameters
    vel = 1.0
    diff = 0.02 / 2.0
    nPt = 1000
    x = np.linspace(0, 1, nPt).reshape(-1, 1)

    # Boundary Conditions
    C_BC1, C_BC2 = 1.0, 0.1
    xb = np.array([0.0, 1.0], dtype=np.float32).reshape(-1, 1)
    cb = np.array([C_BC1, C_BC2], dtype=np.float32).reshape(-1, 1)

    # Analytical Solution
    A = (C_BC2 - C_BC1) / (exp(vel / diff) - 1)
    B = C_BC1 - A
    C_analytical = A * np.exp(vel / diff * x) + B

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Train PINN with exact BCs
    # CHANGE: keep everything exactly as first code
    model = pinn_train(x, xb, cb, vel, diff,
                       batch_size=50,
                       epochs=6000,
                       flag_bc_exact=True,
                       device=device)

    # Evaluate model
    x_tensor = torch.tensor(x, dtype=torch.float32, device=device)
    C_pred = model(x_tensor).detach().cpu().numpy()

    # Plot results
    plt.figure()
    plt.plot(x, C_analytical, 'r--', label='Analytical')
    plt.plot(x, C_pred, 'go', label='PINN Prediction', alpha=0.6)
    plt.xlabel("x")
    plt.ylabel("C(x)")
    plt.legend()
    plt.title("1D Advection-Diffusion PINN Solution (Exact BC)")
    plt.show()


Using device: cpu
Epoch 0, Loss: 8.107598e-01
Epoch 500, Loss: 8.067780e-01
Epoch 1000, Loss: 8.067055e-01
Epoch 1500, Loss: 8.065000e-01
Epoch 2000, Loss: 7.979471e-01
Epoch 2500, Loss: 7.034355e-01
Epoch 3000, Loss: 4.406098e-01
Epoch 3500, Loss: 2.197683e-01
Epoch 4000, Loss: 9.139805e-02
Epoch 4500, Loss: 3.245525e-02
Epoch 5000, Loss: 1.092965e-02
Epoch 5500, Loss: 3.923119e-03
Elapsed time = 51.543469190597534 seconds


RuntimeError: Numpy is not available